In [1]:
!pip install timm fastkaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 9.3 MB/s eta 0:00:00a 0:00:01


In [2]:
import timm
import fastai
from fastai.vision.all  import *
from fastcore.parallel import *
import pandas as pd 
import numpy as np
from fastkaggle import setup_comp

In [3]:
comp = 'paddy-disease-classification'

In [4]:
path = setup_comp(comp, install='fastai "timm>=0.6.2.dev0"')

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-io 0.21.0 requires tensorflow-io-gcs-filesystem==0.21.0, which is not installed.
tensorflow 2.6.4 requires h5py~=3.1.0, but you have h5py 3.7.0 which is incompatible.
tensorflow 2.6.4 requires numpy~=1.19.2, but you have numpy 1.21.6 which is incompatible.
tensorflow 2.6.4 requires tensorboard<2.7,>=2.6.0, but you have tensorboard 2.10.0 which is incompatible.
tensorflow 2.6.4 requires typing-extensions<3.11,>=3.7, but you have typing-extensions 4.1.1 which is incompatible.
tensorflow-transform 1.9.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,!=2.4.*,!=2.5.*,!=2.6.*,!=2.7.*,!=2.8.*,<2.10,>=1.15.5, but you have tensorflow 2.6.4 which is incompatible.
tensorflow-serving-api 2.9.0 requires tensorflow<3,>=2.9.0, but you have tensorflow 2.6.4 which is incompatible.
pandas-profiling 3.1.0 requires mar

In [5]:
trn_path = path/'train_images'
files = get_image_files(trn_path)

In [6]:
df = pd.read_csv(path/'train.csv')

In [7]:
# trn_path = Path("sml")
# resize_images(path/"train_images",dest= trn_path,max_size= 256, recurse= True)

In [8]:
tst_files = get_image_files(path/'test_images').sorted()

In [9]:
def train(arch,size,lr, item=Resize(480, method='squish'),accum=1, epochs=5,finetune = True):
    dls = ImageDataLoaders.from_folder(trn_path, seed=42, valid_pct=0.2, item_tfms=item, 
                                       batch_tfms=aug_transforms(size=size, min_scale=0.75),bs =64//accum)
    cbs = GradientAccumulation(64) if accum else []
    learn = vision_learner(dls, arch, metrics=error_rate,cbs = cbs).to_fp16()

    if finetune:
        learn= learn.fine_tune(epochs, lr)
        return learn.tta(dl = dls.test_dl(tst_files))
    else:
        learn.unfreeze()
        learn.fit_one_cycle(epochs, lr)

In [ ]:
arch = 'convnext_small_in22k'
item = Resize((256,192), method=ResizeMethod.Pad, pad_mode=PadMode.Zeros)
batch = aug_transforms(size=(171,128), min_scale=0.75)
lr=1e-2

In [10]:
import gc
def report_gpu():
    print(torch.cuda.list_gpu_processes())
    gc.collect()
    torch.cuda.empty_cache()

In [12]:
train('convnext_small_in22k',lr =0.01,size= 128, epochs=1, accum=1, finetune=False)
report_gpu()

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_small_22k_224.pth" to /root/.cache/torch/hub/checkpoints/convnext_small_22k_224.pth


epoch,train_loss,valid_loss,error_rate,time
0,2.565921,15.938531,0.952427,02:13


GPU:0
process       2062 uses     4071.000 MB GPU memory


In [ ]:
# learn.lr_find(suggest_funcs=(valley, slide))

In [13]:
res = 640,480

In [14]:
models = {
    'convnext_large_in22k': {
        (Resize(res), (320,224)),
    }, 'vit_large_patch16_224': {
        (Resize(480, method='squish'), 224),
        (Resize(res), 224),
    }, 'swinv2_large_window12_192_22k': {
        (Resize(480, method='squish'), 192),
        (Resize(res), 192),
    }, 'swin_large_patch4_window7_224': {
        (Resize(res), 224),
    }
}

In [15]:
tta_res = []

for arch,details in models.items():
    for item,size in details:
        print('---',arch)
        print(size)
        print(item.name)
        tta_res.append(train(arch,lr=0.1, size =size, item=item, accum=2)) #, epochs=1))
        gc.collect()
        torch.cuda.empty_cache()

--- convnext_large_in22k
(320, 224)
Resize -- {'size': (480, 640), 'method': 'crop', 'pad_mode': 'reflection', 'resamples': (<Resampling.BILINEAR: 2>, <Resampling.NEAREST: 0>), 'p': 1.0}


Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_large_22k_224.pth" to /root/.cache/torch/hub/checkpoints/convnext_large_22k_224.pth


epoch,train_loss,valid_loss,error_rate,time
0,1.610974,1.051600,0.263335,05:35


epoch,train_loss,valid_loss,error_rate,time
0,0.697593,1.435956,0.205670,07:43
1,0.683546,3.238903,0.205190,07:43
2,0.338588,2.983634,0.078808,07:43
3,0.159537,0.137074,0.030274,07:43
4,0.072364,0.109483,0.024988,07:43


AttributeError: 'NoneType' object has no attribute 'tta'

In [ ]:
save_pickle('tta_res.pkl', tta_res)


In [ ]:
tta_prs = first(zip(*tta_res))
tta_prs += tta_prs[1:3]


In [ ]:
avg_pr = torch.stack(tta_prs).mean(0)
avg_pr.shape

**Prediction and Submission**

In [ ]:
#just want to get vocab
dls = ImageDataLoaders.from_folder(trn_path, valid_pct=0.2, item_tfms=Resize(480, method='squish'),
    batch_tfms=aug_transforms(size=224, min_scale=0.75))

In [ ]:
# road to top 3 submission method
idxs = avg_pr.argmax(dim=1)
vocab = np.array(dls.vocab)
ss = pd.read_csv(path/'sample_submission.csv')
ss['label'] = vocab[idxs]
ss.to_csv('subm.csv', index=False)

In [ ]:
#road to top 1 and 2  submission method

In [ ]:
tst_files = get_image_files(path/'test_images').sorted()
tst_dl = dls.test_dl(tst_files)

In [ ]:
probs,_,idxs = learn.get_preds(dl=tst_dl, with_decoded=True)
idxs

In [ ]:
mapping = dict(enumerate(dls.vocab))
results = pd.Series(idxs.numpy(), name="idxs").map(mapping)
results

In [ ]:
ss['label'] = results
ss.to_csv('subm2.csv', index=False)
!head subm2.csv